In [124]:
'''

Project: Neural Network for MHC Peptide Prediction
Class(s): (none) 
Function: Organizes main pipeline execution and testing 

Author: Patrick V. Holec
Date Created: 2/3/2017
Date Updated: 3/20/2017

This is for actual data testing

'''

# standard libraries
import time
import pickle
import collections

# nonstandard libraries
import matplotlib.pyplot as plt
import numpy as np


In [152]:

aas = 'ACDEFGHIKLMNPQRSTVWY_'
threshold = 3


model_files = ['./logs/model_{}.p'.format(i) for i in xrange(10005,10009)]
data_files = ['A12_{}-{}_seqs.txt'.format(i,i+1) for i in xrange(1,5)]

weights,confidences = [],[]


# Acquire sw/pw trained matrices
for model_file in model_files:
    
    model_dict = pickle.load(open(model_file,'rb'))
    
    # Load layers
    W_sw_layer = np.squeeze(model_dict['W_sw'])
    W_pw_layer = np.squeeze(model_dict['W_pw'])
    W_fc_layers = np.squeeze(model_dict['W_fc'])
    b_fc_layers = np.squeeze(model_dict['b_fc'])
    
    # Split FC contributions
    W_fc = np.dot(W_fc_layers[0],np.add(W_fc_layers[1],np.expand_dims(b_fc_layers[0],axis=1)))
    W_sw_weight,W_pw_weight = np.split(W_fc,[W_sw_layer.shape[1]])

    ### Manipulate into standard format ###
    # SW Contribution
    W_sw_weight = np.reshape(np.tile(W_sw_weight,21),(W_sw_weight.size,21)).T
    W_sw = np.multiply(W_sw_layer,W_sw_weight)
    # PW Contribution
    W_pw_weight = np.reshape(np.tile(W_pw_weight,21**2),(W_pw_weight.size,21**2)).T
    W_pw_weight = np.reshape(W_pw_weight,(21,21,W_pw_weight.shape[1]))
    W_pw = np.multiply(W_pw_layer,W_pw_weight)
    
    weights.append((W_sw,W_pw))

    
# Solve for sw/pw contributions
for data_file in data_files:
    
    with open(data_file,'r') as f:
        content = f.readlines()
    sequences = [x.strip() for x in content[1:]] 
    sequences = [x[:x.index(',')] for x in sequences] 
        
    # sw confidence calculations
    sw_confidence = np.zeros(W_sw_weight.shape)
    for i,sample in enumerate(sequences):
        for j,char in enumerate(sample):
            try: sw_confidence[aas.index(char),j] += 1
            except ValueError: raw_input(aas+':'+char)
    
    # pw confidence calculations
    pw_confidence = np.zeros(W_pw_weight.shape)
    for i,sample in enumerate(sequences):
        pair_index = 0
        for j,char1 in enumerate(sample):
            for k,char2 in enumerate(sample[j+1:]):
                pw_confidence[aas.index(char1),aas.index(char2),pair_index] += 1
                pair_index += 1
                
    confidences.append((sw_confidence,pw_confidence))

    
# Reduce the trained models to only show confident outputs
for w,c,n in zip(weights,confidences,data_files):
    
    W_sw,W_pw = w
    C_sw,C_pw = c
    
    # start sw processing
    for i in xrange(W_sw.shape[0]):
        for j in xrange(W_sw.shape[1]):
            if C_sw[i][j] < threshold:
                W_sw[i][j] = -np.inf

    # start pw processing
    for i in xrange(W_pw.shape[0]):
        for j in xrange(W_pw.shape[1]):
            for k in xrange(W_pw.shape[2]):
                if C_pw[i][j][k] < threshold:
                    W_pw[i][j][k] = -np.inf
    
    # so the plan: look for best single pair, keep building from there
    
    total_score = 0.
    accepted_positions = [7]
    accepted_residues = ['-' for i in xrange(W_sw.shape[1])]
    accepted_residues[7] = 'F'
    positions = list(xrange(W_sw.shape[1]))
    

    # sitewise start!
    
    
    # iterative additions
    while len(positions) > len(accepted_positions):
        
        pair_dict = {}
                
        pair_index = 0
        for i in xrange(W_sw.shape[1]): # ind1
            for j in xrange(i+1,W_sw.shape[1]): # ind2
                if (i in accepted_positions) != (j in accepted_positions): #exclusive or
                    
                    if i in accepted_positions:
                        k = aas.index(accepted_residues[i])
                        for l in xrange(W_pw.shape[0]):
                            pair_dict[(i,aas[k],j,aas[l])] = W_sw[l][j]
                            for p in accepted_positions:
                                pair_dict[(i,aas[k],j,aas[l])] += W_pw[k][l][pair_index]  
                    
                    elif j in accepted_positions:
                        l = aas.index(accepted_residues[j])
                        for k in xrange(W_pw.shape[0]): # aa1
                            pair_dict[(i,aas[k],j,aas[l])] = W_sw[k][i]
                            for p in accepted_positions:
                                pair_dict[(i,aas[k],j,aas[l])] += W_pw[k][l][pair_index]  
                            
                pair_index += 1                    

        #print 'Options:',len(pair_dict)
        pair_dict = collections.Counter(pair_dict)
        pair_dict.most_common()
        
        # accept highest scoring option
        [(k,v)] = pair_dict.most_common(1)
        total_score += v
        for ind,res in zip([k[0],k[2]],[k[1],k[3]]):
            #print 'Adding: {}{}'.format(ind,res)
            if not ind in accepted_positions:
                accepted_positions.append(ind)
                accepted_residues[ind] = res
                W_sw[:,ind] = 0
        accepted_positions = list(set(accepted_positions))
        
        print 'Current peptide: {}'.format(''.join(accepted_residues))
    print 'Total score:',total_score
        #print accepted_positions
        
        

    '''
    # merge sw matrix into pw matrix
    pair_index = 0
    for i in xrange(W_sw.shape[1]):
        for j in xrange(i+1,W_sw.shape[1]):
            inds = np.unravel_index(W_pw_score[:,:,pair_index].argmax(),W_pw_score[:,:,pair_index].shape)
            print '{} : {}{} (Score - {})'.format((i+1,j+1),aas[inds[0]],aas[inds[1]],W_pw_score[inds[0],inds[1],pair_index])
            pair_index += 1
    '''        
            
                                    
    
    
    
    #plt.pcolor(W_sw)
    #plt.colorbar()
    #plt.show()
    
        


Current peptide: -----L-F------
Current peptide: -E---L-F------
Current peptide: -E-S-L-F------
Current peptide: -E-S-L-F-----G
Current peptide: -E-S-L-FY----G
Current peptide: -E-STL-FY----G
Current peptide: -E-STLAFY----G
Current peptide: -E-STLAFY---VG
Current peptide: -E-STLAFY--RVG
Current peptide: -EGSTLAFY--RVG
Current peptide: -EGSTLAFY-VRVG
Current peptide: -EGSTLAFYAVRVG
Current peptide: AEGSTLAFYAVRVG
Total score: 19.8467270955
Current peptide: ------FF------
Current peptide: ------FF-A----
Current peptide: -----HFF-A----
Current peptide: -----HFF-A--V-
Current peptide: E----HFF-A--V-
Current peptide: E----HFFRA--V-
Current peptide: E---LHFFRA--V-
Current peptide: E---LHFFRA-IV-
Current peptide: E---LHFFRAQIV-
Current peptide: E-G-LHFFRAQIV-
Current peptide: ENG-LHFFRAQIV-
Current peptide: ENGRLHFFRAQIV-
Current peptide: ENGRLHFFRAQIVG
Total score: 24.3885638714
Current peptide: -R-----F------
Current peptide: AR-----F------
Current peptide: AR-----F----P-
Current peptide: A

In [113]:
print W_sw.shape

(21, 14)
